In [1]:
import cracknuts as cn

In [2]:
g1 = cn.cracker_g1('192.168.0.10')
g1.connect(force_update_bin=True, force_write_default_config=True)

In [3]:
import random
import time
from cracknuts.cracker import serial


cmd_set_aes_enc_key = "01 00 00 00 00 00 00 10"
cmd_aes_enc = "01 02 00 00 00 00 00 10"

aes_key = "11 22 33 44 55 66 77 88 99 00 aa bb cc dd ee ff"
aes_data_len = 16

sample_length = 20000

def init(cracker):
    
    cracker.nut_clock_freq('8M')
    cracker.nut_clock_enable()
    cracker.nut_voltage(3.3)
    cracker.nut_voltage_enable()
    cracker.uart_io_enable()
    cracker.osc_sample_clock('48m')
    cracker.osc_sample_length(sample_length)
    cracker.osc_trigger_source('N')
    # cracker.osc_analog_enable('A')
    # cracker.osc_analog_gain('A', 1)
    # cracker.osc_analog_disable('B')
    cracker.osc_analog_gain('B', 10)
    cracker.osc_trigger_level(0)
    cracker.osc_trigger_mode('E')
    cracker.osc_trigger_edge('U')
    cracker.uart_config(
        baudrate=serial.Baudrate.BAUDRATE_115200, 
        bytesize=serial.Bytesize.EIGHTBITS, 
        parity=serial.Parity.PARITY_NONE, 
        stopbits=serial.Stopbits.STOPBITS_ONE)

    time.sleep(2)
    cmd = cmd_set_aes_enc_key + aes_key
    status, ret = cracker.uart_transmit_receive(cmd, timeout=1000, rx_count=6)

def do(cracker, count):
    # plaintext_data = random.randbytes(aes_data_len)
    plaintext_data = bytes.fromhex('AA BB CC DD EE FF 00 11 22 33 44 55 66 77 88 99')
    tx_data = bytes.fromhex(cmd_aes_enc.replace(' ', '')) + plaintext_data
    status, ret = cracker.uart_transmit_receive(tx_data, rx_count= 6 + aes_data_len, is_trigger=True)
    print(ret.hex(' ') if ret else 'None')
    return {
        "plaintext": plaintext_data,
        "ciphertext": b'',
        "key": bytes.fromhex(aes_key)
    }


acq = cn.simple_glitch_acq(g1, init, do)

p = cn.show_panel(acq)
p

In [4]:
g1.glitch_vcc_normal(3.3)
g1.glitch_gnd_normal(0.0)
g1.glitch_vcc_config(wait=1, level=1.5, length=20, delay=10, repeat=2)
g1.nut_voltage_enable()

(0, None)

In [5]:
g1.glitch_vcc_force()

(0, None)

In [6]:
g1.glitch_vcc_normal(3.3)
g1.glitch_gnd_normal(0.0)
g1.glitch_gnd_config(wait=1, level=1.5, length=20, delay=10, repeat=2)
g1.nut_voltage_enable()

(0, None)

In [9]:
g1.glitch_gnd_force()

(0, None)